In [1]:
# Для считывания PDF
import PyPDF2
# Для анализа структуры PDF и извлечения текста
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
# Для извлечения текста из таблиц в PDF
import pdfplumber
# Для извлечения изображений из PDF
from PIL import Image
from pdf2image import convert_from_path
# Для удаления дополнительно созданных файлов
import os
# Для сохранение промежуточных результатов
import joblib

In [2]:
# Создаём функцию для извлечения текста

def text_extraction(element):
    # Извлекаем текст из вложенного текстового элемента
    line_text = element.get_text()
    
    # Находим форматы текста
    # Инициализируем список со всеми форматами, встречающимися в строке текста
    line_formats = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            # Итеративно обходим каждый символ в строке текста
            for character in text_line:
                if isinstance(character, LTChar):
                    # Добавляем к символу название шрифта
                    line_formats.append(character.fontname)
                    # Добавляем к символу размер шрифта
                    line_formats.append(character.size)
    # Находим уникальные размеры и названия шрифтов в строке
    format_per_line = list(set(line_formats))
    
    # Возвращаем кортеж с текстом в каждой строке вместе с его форматом
    return (line_text, format_per_line)

In [3]:
pdf_path = "lunar_articles_list.pdf"

# создаём объект файла PDF
pdfFileObj = open(pdf_path, 'rb')
# создаём объект считывателя PDF
pdfReaded = PyPDF2.PdfReader(pdfFileObj)

In [4]:
# Создаём словарь для извлечения текста из каждого изображения
text_per_page = {}
# Извлекаем страницы из PDF
for pagenum, page in enumerate(extract_pages(pdf_path)):
    
    # Инициализируем переменные, необходимые для извлечения текста со страницы
    pageObj = pdfReaded.pages[pagenum]
    page_text = []
    line_format = []
    text_from_images = []
    text_from_tables = []
    page_content = []
    # Инициализируем количество исследованных таблиц
    table_num = 0
    first_element= True
    table_extraction_flag= False
    # Открываем файл pdf
    pdf = pdfplumber.open(pdf_path)
    # Находим исследуемую страницу
    page_tables = pdf.pages[pagenum]
    # Находим количество таблиц на странице
    tables = page_tables.find_tables()


    # Находим все элементы
    page_elements = [(element.y1, element) for element in page._objs]
    # Сортируем все элементы по порядку нахождения на странице
    page_elements.sort(key=lambda a: a[0], reverse=True)

    # Находим элементы, составляющие страницу
    for i,component in enumerate(page_elements):
        # Извлекаем положение верхнего края элемента в PDF
        pos= component[0]
        # Извлекаем элемент структуры страницы
        element = component[1]
        
        # Проверяем, является ли элемент текстовым
        if isinstance(element, LTTextContainer):
            # Проверяем, находится ли текст в таблице
            if table_extraction_flag == False:
                # Используем функцию извлечения текста и формата для каждого текстового элемента
                (line_text, format_per_line) = text_extraction(element)
                # Добавляем текст каждой строки к тексту страницы
                page_text.append(line_text)
                # Добавляем формат каждой строки, содержащей текст
                line_format.append(format_per_line)
                page_content.append(line_text)
            else:
                # Пропускаем текст, находящийся в таблице
                pass

    # Создаём ключ для словаря
    dctkey = 'Page_'+str(pagenum)
    # Добавляем список списков как значение ключа страницы
    text_per_page[dctkey]= [page_text, line_format, text_from_images,text_from_tables, page_content]


In [5]:
# Закрываем объект файла pdf
pdfFileObj.close()

# Удаляем созданные дополнительные файлы
# os.remove('cropped_image.pdf')
# os.remove('PDF_image.png')

# Удаляем содержимое страницы
result = ''.join(text_per_page['Page_0'][4])
# print(result)

In [6]:
page_keys = list(text_per_page.keys())
pages = [text_per_page[pgkey][0] for pgkey in page_keys]

In [7]:
lines = []
for page in pages:
    for line in page:
        if line.strip() == '':
            continue
        lines.append(line.strip())
lines = lines[2:]

In [8]:
from dataclasses import dataclass

@dataclass
class Author:
    Name: str
    Initials: str

    def __hash__(self):
        return (self.Name + self.Initials).__hash__()
    
    def __repr__(self):
        return self.Name + ' ' + self.Initials

@dataclass
class PubYear:
    Year: int
    Letter: str

@dataclass
class Affiliation:
    Uni: str
    Year: int
    
    def __hash__(self):
        return f'{self.Uni} ({self.Year})'.__hash__()
    
    def __le__(self, other: 'Affiliation'):
        if self.Year < other.Year:
            return True
        if self.Year > other.Year:
            return False
        return self.Uni < other.Uni

In [9]:
import re

def parse_line_raw(reference: str):
    reference = reference.replace('\n', '')
    
    # Pattern to match authors, year, title, and the rest
    pattern = r"^(.*?)(\(\d{4}[a-z]?\))\s+(.*?)(\.\s.*)$"
    match = re.match(pattern, reference)
    
    if not match:
        return None  # Return None if the pattern does not match
    
    authors = match.group(1).strip()
    year = match.group(2).strip("()")
    title = match.group(3).strip()
    rest = match.group(4).strip()
    
    return [authors, year, title, rest]

def split_authors(authors: str) -> list[Author]:
    author_list = []
    
    # Split the authors by commas and "and"
    author_entries = re.split(r'\s*(?:,\s*|\s+and\s+)\s*', authors)
    
    for entry in author_entries:
        # Split by spaces to separate names and initials
        name_parts = entry.strip().split()
        
        # The last part is likely the last name, others are initials
        name = ' '.join(name_parts[:-1])  # First and middle names
        initials = name_parts[-1]  # Last part is typically the initials
        
        author_list.append(Author(Name=name, Initials=initials))
    
    return author_list

def parse_line(line: str) -> tuple[list[Author], PubYear, str, str]:
    authors, year, title, rest = parse_line_raw(line)
    authors = split_authors(authors)
    year = PubYear(Year=int(year[:4]), Letter=year[4:])
    return (authors, year, title, rest)

In [10]:
parsed = []
skipped = []
for i, ch in enumerate(lines):
    ps = parse_line_raw(ch)
    if ps is None or len(ps[0]) == 0:
        skipped.append((i, ch))
    else:
        parsed.append(parse_line(ch))

print(f'Processed {len(parsed)}/{len(lines)} entries, {len(skipped)} skipped')

Processed 3819/3845 entries, 26 skipped


In [11]:
all_authors = set()
for p in parsed:
    for a in p[0]:
        all_authors.add(a)

joblib.dump(all_authors, 'authors.pkl')

print(f'{len(parsed)} articles have {len(all_authors)} unique authors, with a number of {len(all_authors) / len(parsed):.1f} authors per publication')

3819 articles have 3096 unique authors, with a number of 0.8 authors per publication


In [12]:
a1 = list(all_authors)[0]
a1

Taylor H.C.

In [109]:
from scholarly import scholarly

# Search for the author by name
search_query = scholarly.search_author(a1.__repr__())
for au in search_query:
    au_data = scholarly.fill(au)
    print(au_data)

{'container_type': 'Author', 'filled': ['basics', 'indices', 'counts', 'coauthors', 'publications', 'public_access'], 'source': <AuthorSource.SEARCH_AUTHOR_SNIPPETS: 'SEARCH_AUTHOR_SNIPPETS'>, 'scholar_id': 'kwbEUvYAAAAJ', 'url_picture': 'https://scholar.google.com/citations?view_op=medium_photo&user=kwbEUvYAAAAJ', 'name': 'Joshua D.M. Shaw', 'affiliation': 'Lecturer in Law at the University of Kent', 'email_domain': '@kent.ac.uk', 'interests': ['human tissue law', 'law and dead bodies', 'legal theory', 'medical law', 'socio-legal studies'], 'citedby': 99, 'organization': 15489720745136061206, 'homepage': 'https://www.kent.ac.uk/law/people/4977/Shaw-Joshua', 'citedby5y': 70, 'hindex': 4, 'hindex5y': 4, 'i10index': 4, 'i10index5y': 3, 'cites_per_year': {2013: 1, 2014: 1, 2015: 5, 2016: 5, 2017: 12, 2018: 5, 2019: 4, 2020: 7, 2021: 6, 2022: 6, 2023: 25, 2024: 16}, 'coauthors': [{'container_type': 'Author', 'filled': [], 'scholar_id': 't09FPNkAAAAJ', 'source': <AuthorSource.CO_AUTHORS_LIS

In [111]:
def get_affiliations(author_scholar_ref: dict) -> set[Affiliation]:
    if 'publications' in author_scholar_ref['filled']:
        affs = set()
        affs.add(Affiliation(author_scholar_ref['affiliation'], 2024))
        for pub in author_scholar_ref['publications']:
            pub_year = pub['pub_year'] if 'pub_year' in pub else 0
            if 'affiliation' in pub:
                aff = pub['affilition']
                affs.add(Affiliation(aff, pub_year))
        return affs
    return set()

In [113]:
from tqdm import tqdm

In [114]:
all_affiliations = set()
for au in tqdm(all_authors):
    search_query = scholarly.search_author(au.__repr__())
    for au_ref in search_query:
        au_data = scholarly.fill(au_ref)
        affs = get_affiliations(au_data)
        all_affiliations = all_affiliations.union(affs)

  0%|          | 1/3096 [00:18<16:17:21, 18.95s/it]


KeyboardInterrupt: 

In [ ]:
all_unis = set()

for aff in all_affiliations:
    uni = aff.Uni
    all_unis.append(uni)